## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [7]:
for name, param in model.named_parameters():
    print(f'{name}, {param.shape}')

conv1.weight, torch.Size([64, 3, 7, 7])
bn1.weight, torch.Size([64])
bn1.bias, torch.Size([64])
layer1.0.conv1.weight, torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight, torch.Size([64])
layer1.0.bn1.bias, torch.Size([64])
layer1.0.conv2.weight, torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight, torch.Size([64])
layer1.0.bn2.bias, torch.Size([64])
layer1.1.conv1.weight, torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight, torch.Size([64])
layer1.1.bn1.bias, torch.Size([64])
layer1.1.conv2.weight, torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight, torch.Size([64])
layer1.1.bn2.bias, torch.Size([64])
layer2.0.conv1.weight, torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight, torch.Size([128])
layer2.0.bn1.bias, torch.Size([128])
layer2.0.conv2.weight, torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight, torch.Size([128])
layer2.0.bn2.bias, torch.Size([128])
layer2.0.downsample.0.weight, torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight, torch.Size([128])
layer2.0.downsample.1.bias, torch.Size([128])
l

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [8]:
input_ = torch.randn(1, 3, 128, 128)

In [9]:
for param in model.parameters():
    param.requires_grad=False

In [10]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [11]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [18]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(12,10)
        self.BN = nn.BatchNorm1d(10)
        self.linear2 = nn.Linear(10,10)
        self.linear3 = nn.Linear(10,3)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        for i in range(10):
            x = F.relu(self.linear2(x))
            x = F.relu(self.BN(x))
        return F.relu(self.linear3(x))

In [19]:
model = Model()

In [20]:
model

Model(
  (linear1): Linear(in_features=12, out_features=10, bias=True)
  (BN): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
)

In [28]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [29]:
outupt

tensor([[0.5072, 0.0000, 0.0000],
        [0.0000, 0.4601, 0.0634],
        [0.0000, 0.0000, 0.0150],
        [0.0000, 0.0000, 0.0411],
        [0.0000, 0.0000, 0.0411],
        [0.0000, 0.0000, 0.0411],
        [0.0000, 0.0000, 1.5935],
        [0.0000, 0.0000, 0.0411],
        [0.0000, 0.0000, 0.0411],
        [0.0000, 0.0000, 0.0411]], grad_fn=<ReluBackward0>)